In [1]:
import os
import numpy as np
import pandas as pd
import random as rn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn import tree
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
import seaborn as sns

Data nejsou součástí protože se často jedné o malwer!!

In [2]:
def load_premmission(directory):
    list_of_premm = [ ]
    list_of_premm_sep = [ ]
    my_dir = os.listdir(directory)
    for apk in my_dir:
        apk_f = directory+ apk
        premm = os.popen(f"aapt d permissions '{apk_f}'").read()
        premm_sorted  = premm.split('\n')
        step2 = []
        for row in range(len(premm_sorted)):
            step = premm_sorted[row].partition("name=")[2]
            list_of_premm.append(step)
            list_of_premm = list(filter(None, list_of_premm))
            step2.append(step)
            step2 = list(filter(None, step2))
        list_of_premm_sep.append(step2)
        list_of_premm_sep = list(filter(None, list_of_premm_sep))
    for i in range(len(list_of_premm)):
        list_of_premm[i] = list_of_premm[i].strip("'")
    list_of_premm = np.unique(list_of_premm).tolist()
    for ls in range(len(list_of_premm_sep)):
        for ele in range(len(list_of_premm_sep[ls])):
            list_of_premm_sep[ls][ele]= list_of_premm_sep[ls][ele].strip("'")
        
    return list_of_premm, list_of_premm_sep

# Infi

In [3]:
directory_infi = '/home/kali/Desktop/apk/infi_test/'
list_of_premm_infi, list_of_premm_sep_infi = load_premmission(directory_infi)

# Beign

In [4]:
directory_beign = '/home/kali/Desktop/apk/beign_test1_1/'
list_of_premm_beign, list_of_premm_sep_beign = load_premmission(directory_beign)

subsets for graphs

In [5]:
directory_beign1_2 = '/home/kali/Desktop/apk/beign_test1_2/'
list_of_premm_beign1_2, list_of_premm_sep_beign1_2 = load_premmission(directory_beign1_2)
directory_beign1_3 = '/home/kali/Desktop/apk/beign_test1_3/'
list_of_premm_beign1_3, list_of_premm_sep_beign1_3 = load_premmission(directory_beign1_3)
directory_beign1_4 = '/home/kali/Desktop/apk/beign_test1_4/'
list_of_premm_beign1_4, list_of_premm_sep_beign1_4 = load_premmission(directory_beign1_4)

ERROR: dump failed because no AndroidManifest.xml found


In [6]:
over_b = list_of_premm_sep_beign + list_of_premm_sep_beign1_2 +list_of_premm_sep_beign1_3 + list_of_premm_sep_beign1_4

# Join infi-beign + empty feature vector 

In [7]:
def feat_vec(infi_overall,beign_overall,infi_sep,beign_sep):
    over_all_sep = infi_sep+beign_sep
    over_all_premm = infi_overall + beign_overall
    over_all_premm = np.unique(over_all_premm).tolist()
    rn.shuffle(over_all_premm)
    number_of_rows = len(infi_sep)+len(beign_sep)
    empty_feat_vec = np.zeros((number_of_rows,len(over_all_premm)),int)
    return over_all_premm, empty_feat_vec, over_all_sep

In [8]:
over_all_premm, empty_feat_vec, over_all_sep = feat_vec(list_of_premm_infi,list_of_premm_beign,list_of_premm_sep_infi,list_of_premm_sep_beign)

In [9]:
def fill_empt_vct(over_all_premm,over_all_sep,empty_feat_vec):
    for row in range(len(over_all_sep)):
        for col in range(len(over_all_premm)):
            if over_all_premm[col] in over_all_sep[row]:
                empty_feat_vec[row,col] = 1
    return empty_feat_vec   

In [10]:
fill_vct = fill_empt_vct(over_all_premm,over_all_sep,empty_feat_vec)

In [11]:
def Targets(infi_sep,beign_sep,fill_vct):
    targets = [1]*len(infi_sep)
    targets += [0]*len(beign_sep)
    arr_x, arr_y = shuffle(fill_vct,targets)
    return arr_x, arr_y

In [12]:
arr_x,arr_y = Targets(list_of_premm_sep_infi,list_of_premm_sep_beign,fill_vct)

# Data classification

In [13]:
train_X,test_X,train_y,test_y = train_test_split(arr_x,arr_y)

MLP methods:
- lbfgs
- adam

In [14]:
#Parameters
n_of_hidd_lay = 2
n_feat = np.shape(arr_x)[1]
n_of_hiddneur =  int(n_of_hidd_lay*(np.ceil(np.sqrt(n_feat*1))))

In [15]:
mlp_classifier_lbfgs = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(n_of_hiddneur, n_of_hiddneur), random_state=1)
mlp_classifier_lbfgs.fit(train_X,train_y)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(30, 30), random_state=1,
              solver='lbfgs')

In [16]:
mlp_classifier_lbfgs.score(test_X,test_y)

0.9090909090909091

In [17]:
mlp_classifier_adam = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(n_of_hiddneur, n_of_hiddneur), random_state=1,max_iter=500)
mlp_classifier_adam.fit(train_X,train_y)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(30, 30), max_iter=500,
              random_state=1)

In [18]:
mlp_classifier_adam.score(test_X,test_y)

0.7272727272727273

SWM methods:
- rbf
- poly

In [19]:
#Parameters
C = 1.0 
h = 0.02

In [20]:
SVC_classifier_rbf = SVC(kernel='rbf', gamma=0.7, C=C)
SVC_classifier_rbf.fit(train_X,train_y)

SVC(gamma=0.7)

In [21]:
SVC_classifier_rbf.score(test_X,test_y)

0.6363636363636364

In [22]:
SVC_classifier_poly = SVC(kernel='poly', gamma=0.7, C=C)
SVC_classifier_poly.fit(train_X,train_y)

SVC(gamma=0.7, kernel='poly')

In [23]:
SVC_classifier_poly.score(test_X,test_y)

0.9090909090909091

Decision Trees
- regression
- classification

In [24]:
TREE_classifier = tree.DecisionTreeClassifier()
TREE_classifier.fit(train_X,train_y)

DecisionTreeClassifier()

In [25]:
TREE_classifier.score(test_X,test_y)

0.8181818181818182

In [26]:
TREE_regression = tree.DecisionTreeRegressor()
TREE_regression.fit(train_X,train_y)

DecisionTreeRegressor()

In [27]:
TREE_regression.score(test_X,test_y)

0.2666666666666666

# Cross-validation

MLP lbfgs

In [28]:
mlp_classifier_lbfgsCS = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(n_of_hiddneur, n_of_hiddneur), random_state=1,max_iter=350)
scores_lbfgsCS = cross_val_score(mlp_classifier_lbfgsCS, arr_x, arr_y, cv=5)
scores_lbfgsCS

array([0.88888889, 0.88888889, 0.66666667, 0.625     , 0.875     ])

In [29]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores_lbfgsCS.mean(), scores_lbfgsCS.std()))

0.79 accuracy with a standard deviation of 0.12


In [30]:
y_pred_lbfgsCS = cross_val_predict(mlp_classifier_lbfgsCS, arr_x, arr_y, cv=5)
con_mat = confusion_matrix(arr_y, y_pred_lbfgsCS) #TP,FP / FN,TN
con_DF = pd.DataFrame(con_mat)
cm = sns.light_palette("green", as_cmap=True)
con_DF.style.background_gradient(cmap=cm)

,0,1
0,18,5
1,4,16


MLP Adam

In [31]:
mlp_classifier_adamCS = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(n_of_hiddneur, n_of_hiddneur), random_state=1,max_iter=350)
scores_adamCS = cross_val_score(mlp_classifier_adamCS, arr_x, arr_y, cv=5)
scores_adamCS

array([0.55555556, 0.66666667, 0.88888889, 0.75      , 0.75      ])

In [32]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores_adamCS.mean(), scores_adamCS.std()))

0.72 accuracy with a standard deviation of 0.11


In [33]:
y_pred_adamCS = cross_val_predict(mlp_classifier_adamCS, arr_x, arr_y, cv=5)
con_mat = confusion_matrix(arr_y, y_pred_adamCS) #TP,FP / FN,TN
con_DF1 = pd.DataFrame(con_mat)
cm = sns.light_palette("green", as_cmap=True)
con_DF1.style.background_gradient(cmap=cm)

,0,1
0,19,4
1,8,12


SVC rbf

In [34]:
SVC_classifier_rbfCS = SVC(kernel='rbf', gamma=0.7, C=C)
scores_rbfCS = cross_val_score(SVC_classifier_rbfCS, arr_x, arr_y, cv=5)
scores_rbfCS

array([0.66666667, 0.77777778, 0.77777778, 0.625     , 0.625     ])

In [35]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores_rbfCS.mean(), scores_rbfCS.std()))

0.69 accuracy with a standard deviation of 0.07


In [36]:
y_pred_rbfCS = cross_val_predict(SVC_classifier_rbfCS, arr_x, arr_y, cv=5)
con_mat = confusion_matrix(arr_y, y_pred_rbfCS) #TP,FP / FN,TN
con_DF2 = pd.DataFrame(con_mat)
cm = sns.light_palette("green", as_cmap=True)
con_DF2.style.background_gradient(cmap=cm)

,0,1
0,23,0
1,13,7


SVC poly

In [37]:
SVC_classifier_polyCS = SVC(kernel='poly', gamma=0.7, C=C)
scores_polyCS = cross_val_score(SVC_classifier_polyCS, arr_x, arr_y, cv=5)
scores_polyCS

array([0.55555556, 1.        , 0.77777778, 0.625     , 0.875     ])

In [38]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores_polyCS.mean(), scores_polyCS.std()))

0.77 accuracy with a standard deviation of 0.16


In [39]:
y_pred_polyCS = cross_val_predict(SVC_classifier_polyCS, arr_x, arr_y, cv=5)
con_mat = confusion_matrix(arr_y, y_pred_polyCS) #TP,FP / FN,TN
con_DF3 = pd.DataFrame(con_mat)
cm = sns.light_palette("green", as_cmap=True)
con_DF3.style.background_gradient(cmap=cm)

,0,1
0,20,3
1,7,13


In [40]:
TREE_classifierCS = tree.DecisionTreeClassifier()
scores_TREE_classifierCS = cross_val_score(TREE_classifierCS, arr_x, arr_y, cv=5)
scores_TREE_classifierCS

array([0.66666667, 0.77777778, 0.66666667, 0.5       , 0.75      ])

In [41]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores_TREE_classifierCS.mean(), scores_TREE_classifierCS.std()))

0.67 accuracy with a standard deviation of 0.10


In [42]:
y_pred_TREE_classifierCS = cross_val_predict(TREE_classifierCS, arr_x, arr_y, cv=5)
con_mat = confusion_matrix(arr_y, y_pred_TREE_classifierCS) #TP,FP / FN,TN
con_DF4 = pd.DataFrame(con_mat)
cm = sns.light_palette("green", as_cmap=True)
con_DF4.style.background_gradient(cmap=cm)

,0,1
0,17,6
1,6,14


In [43]:
TREE_regressionCS = tree.DecisionTreeRegressor()
scores_TREE_regressionCS = cross_val_score(TREE_regressionCS, arr_x, arr_y, cv=5)
scores_TREE_regressionCS

array([-0.35      ,  0.1       , -1.        ,  0.        , -1.66666667])

In [44]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores_TREE_regressionCS.mean(), scores_TREE_regressionCS.std()))

-0.58 accuracy with a standard deviation of 0.66
